In [1]:
import psycopg2
import requests
from prettytable import PrettyTable as pt
import re
import datetime

In [5]:
class library():
    availability = [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True]
    
    #To enable connection with the database
    def connection(self):
        con=psycopg2.connect(host='localhost',database='project_lib',user='postgres',password='karthi')
        return con
    
    
    #To check the User Credentials
    def login(self):
        res=[]
        username = input("Enter the username:")
        password = input("Enter the password:")
        if(self.connection()):
            con = self.connection()
            cur = con.cursor()
            sql = "select * from {} where user_name='{}' ".format('login',username)
            cur.execute(sql)
            data = cur.fetchall()
            for d in data:
                res.append(d)
            if(res[0][2]==password):
                return True
            else:
                return False
    
    
    #To view the list of books
    def view_books(self):
        if(self.login()):
            url = 'https://61d05c8fcd2ee50017cc9846.mockapi.io/Expense'
            response = requests.get(url)
            data = response.json()
            table = pt(["book_name","author_name","language","onloan","id"])
            for i in range(0,len(data)):
                table.add_row(data[i].values())
            print("List of books",table)
    
    
    #To issue book to a person
    def issue_book(self,book_name):
        if(self.login()):
            url = 'https://61d05c8fcd2ee50017cc9846.mockapi.io/Expense'
            response = requests.get(url)
            data = response.json()
        
            for i in range(0,len(data)):
                dic= data[i].values()
                l = list(dic)
            
                if(l[0] == book_name):
                    if(self.availability[i]):
                        print("Book is Available!!")
                        self.update_db(book_name)
                        self.availability[i]=False
                    else:
                        print("Sorry the book isn't available!!")
                else:
                    pass
    
    
    #To check whether the given mobile number is a valid one!
    def indian_mobile_number(self,mobile):
        result = re.match(r"^[98765][0-9]{9}",mobile)
        if(result):
            return True
        else:
            return False
    
    
    #To check whether the given Email is a valid one!
    def email_check(self,email):
        result = re.match(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',email)
        if(result):
            return True
        else:
            return False
    
    #To update the table of issued list
    def update_db(self,book_name):
        con = self.connection()
        cur = con.cursor()
        current_time = datetime.datetime.now()
        while True:
            
            name = input("Enter the name:")
            email = input("Enter the email:")
            aadhar = int(input("Enter the aadhar number:"))
            mobile = int(input("Enter the mobile number:"))
            if(self.email_check and self.indian_mobile_number):
                break
            else:
                print("Enter valid details")
        sql1 = "select * from books_issued;"
        cur.execute(sql1)
        d=cur.fetchall()
        d = len(d)
        _id = d+1      
            
        sql2 = "insert into books_issued (id,name,mail,aadhar,book_name,time_spand,mobile) values({},'{}','{}','{}','{}','{}','{}');".format(_id,name,email,aadhar,book_name,current_time,mobile)
        
        cur.execute(sql2)
        print("Data added to books issued list")
        con.commit()
        con.close()
    
    #To display the list of books issued
    def display_books_issued(self):
        if(self.login()):
            con = self.connection()
            cur = con.cursor()
            sql = "select * from books_issued"
            cur.execute(sql)
            data=cur.fetchall()
            table = pt(['id','name','mail','book_name','time_spand','mobile','aadhar'])
            for d in data:
                table.add_row(d)
            print("List of Books Issued are:",table)
    
    #Function to return the Book
    def return_book(self,book_name):
        if(self.login()):
            con = self.connection()
            cur = con.cursor()
            sql1 = "select id from books_issued where book_name='{}'".format(book_name)
            cur.execute(sql1)
            d = cur.fetchall()
            _id=d[0][0]
            sql = "delete from books_issued where book_name='{}'".format(book_name)
            cur.execute(sql)
            con.commit()
            self.availability[_id-1] = True
            printf("Book moved to available list!!")
            con.close()
    
        
    
            
        
        
            
            
            
            

In [3]:
l = library()

In [4]:
while True:
    n=int(input(""" Enter 1 to View list of Books
                Enter 2 to Issue Book
                Enter 3 to Display the Books issued
                Enter 4 to Return book
                Enter 0 to Exit"""))
    if(n==1):
        l.view_books()
    elif(n==2):
        book_name=input("Enter the Book Name:")
        l.issue_book(book_name)
    elif(n==3):
        l.display_books_issued()
    elif(n==4):
        book_name=input("Enter the Book Name:")
        l.return_book(book_name)
    elif(n==0):
        print("Thank you!!")
        break
        
        
    

 Enter 1 to View list of Books
                Enter 2 to Issue Book
                Enter 3 to Display the Books issued
                Enter 4 to Return book
                Enter 0 to Exit1
Enter the username:admin2
Enter the password:admin2pw
+---------------------------------+--------------------+----------+--------+----+
|            book_name            |    author_name     | language | onloan | id |
+---------------------------------+--------------------+----------+--------+----+
|             The Guide           |    R.K. NARAYAN    | English  | False  | 1  |
|        A Tiger for Malgud       |    R.K. NARAYAN    | English  | False  | 2  |
|       The English Teacher       |    R.K. NARAYAN    | English  | False  | 3  |
|           The Village           |   MULK RAJ ANAND   | English  | False  | 4  |
|              Coolie             |   MULK RAJ ANAND   | English  | False  | 5  |
|         Train to Pakistan       |  KHUSHWANT SINGH   | English  | False  | 6  |
|       The Roo